In [2]:
import pandas as pd
import osmnx as ox
import numpy as np
import json
import geopandas as gpd
from shapely.geometry import Point

import sys
sys.path.append('../') 
from src.graph.create_osmnx_graph import OSMnxGraph 


/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
cities_pl = ['Warszawa', 'Kraków', 'Wrocław', 'Poznań', 'Szczecin']

all_nodes_pl = pd.DataFrame()
all_edges_pl = pd.DataFrame()

for city in cities_pl:
    G =ox.graph.graph_from_place(f"{city}, Poland", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_pl = pd.concat([all_nodes_pl, gdf_nodes])
    all_edges_pl = pd.concat([all_edges_pl, gdf_edges])


In [ ]:
cities_usa = ['Seattle', 'New York City']

all_nodes_usa = pd.DataFrame()
all_edges_usa = pd.DataFrame()

for city in cities_usa:
    G =ox.graph.graph_from_place(f"{city}, United States", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_usa = pd.concat([all_nodes_usa, gdf_nodes])
    all_edges_usa = pd.concat([all_edges_usa, gdf_edges])

In [ ]:
all_attributes = {}
for col in ['highway', 'access', 'junction', 'bridge', 'tunnel']:
    edges_usa = all_edges_usa[col].dropna().unique()
    edges_pl = all_edges_pl[col].dropna().unique()
    all_edges = set(np.concatenate((edges_pl, edges_usa)))
    all_attributes[f'edges_{col}'] = list(all_edges)

     
nodes_highway_usa = all_nodes_usa['highway'].dropna().unique()
nodes_highway_pl = all_nodes_pl['highway'].dropna().unique()   
all_nodes_highway = set(np.concatenate((nodes_highway_pl, nodes_highway_usa)))
all_attributes['nodes_highway'] = list(all_nodes_highway)

with open('../data/osmnx_attributes.json', 'w') as f:
    json.dump(all_attributes, f)
    

In [17]:
with open("../data/osmnx_attributes.json") as f:
    all_attributes = json.load(f)
    
num_edges_attrs = 0
for key in all_attributes.keys():
    if 'edges' in key:
        num_edges_attrs += len(all_attributes[key])
        
print('num of nodes attrs', len(all_attributes['nodes_highway']))
print('num of edges attrs', num_edges_attrs)

num of nodes attrs 21
num of edges attrs 37


#### Seattle

In [2]:
data = pd.read_csv('../data/downstream_tasks/airbnb/seattle/airbnb_seattle_cleaned.csv')
gdf_airbnb_seattle = gpd.GeoDataFrame(
            data.drop(["latitude", "longitude"], axis=1),
            geometry=gpd.points_from_xy(x=data["longitude"], y=data["latitude"]),
            crs="EPSG:4326",
        )

gdf_airbnb_seattle['airbnb'] = 1

In [3]:
G =ox.graph.graph_from_place("Seattle, United States", network_type="drive", simplify=False)
gdf_nodes_seattle, gdf_edges_seattle = ox.graph_to_gdfs(G)

In [4]:
graph_embedder = OSMnxGraph(gdf_airbnb_seattle, gdf_nodes_seattle, gdf_edges_seattle, y_column_name='airbnb')

osmnx_nodes = graph_embedder.get_node_attrs()
display(osmnx_nodes)
print(len(osmnx_nodes.columns.unique()))
print(osmnx_nodes.columns.unique())
osmnx_nodes.to_csv('../data/osmnx/seattle_osmnx_nodes.csv', index=False)

osmnx_edges = graph_embedder.get_edge_attrs()
display(osmnx_edges)
print(len(osmnx_edges.columns.unique()))
print(osmnx_edges.columns.unique())
osmnx_edges.to_csv('../data/osmnx/seattle_osmnx_edges.csv', index=False)



/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,bus_stop,crossing,give_way,milestone,mini_roundabout,motorway_junction,red_light_camera,speed_camera,stop,...,traffic_signals,traffic_signals;crossing,turning_circle,turning_loop,street_lamp,proposed,stop;crossing,priority,construction,passing_place
osmid,,,,,,,,,,,,,,,,,,,,,
29445653,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445655,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445656,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445657,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445659,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11919304501,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11919304502,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11919304503,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


22
Index(['street_count', 'bus_stop', 'crossing', 'give_way', 'milestone',
       'mini_roundabout', 'motorway_junction', 'red_light_camera',
       'speed_camera', 'stop', 'stop_line', 'toll_gantry', 'traffic_signals',
       'traffic_signals;crossing', 'turning_circle', 'turning_loop',
       'street_lamp', 'proposed', 'stop;crossing', 'priority', 'construction',
       'passing_place'],
      dtype='object')


/Users/jjaniak/Documents/studia/projekt/gradient/notebooks/../src/graph/create_osmnx_graph.py:104: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  war

oneway  lanes maxspeed  reversed  length  \
u           v           key                                               
29445653    4433425415  0   0       1      2       97         0   20.39   
29445655    4433425410  0   0       1      2       97         0   12.41   
29445656    4433425404  0   0       1      2       97         0   14.23   
29445657    1864943659  0   0       1      2       97         0   14.65   
29445659    4433425403  0   0       1      2       97         0   12.50   
...                               ...    ...      ...       ...     ...   
11919304502 11919304503 0   0       0      2       40         1   11.38   
11919304503 11919304502 0   0       0      2       40         0   11.38   
            6585419248  0   0       0      2       40         1    4.09   
11920190240 11920190241 0   0       1      4       40         0   12.28   
11920190241 4273638466  0   0       1      4       40         0   11.09   

                               width  highway_busway  highway_living_street  \
u           v           key                                                   
29445653    4433425415  0   0  12.26               0                      0   
29445655    4433425410  0   0  12.26               0                      0   
29445656    4433425404  0   0  12.26               0                      0   
29445657    1864943659  0   0  12.26               0                      0   
29445659    4433425403  0   0  12.26               0                      0   
...                              ...             ...                    ...   
11919304502 11919304503 0   0  12.26               0                      0   
11919304503 11919304502 0   0  12.26               0                      0   
            6585419248  0   0  12.26               0                      0   
11920190240 11920190241 0   0  12.26               0                      0   
11920190241 4273638466  0   0  12.26               0                      0   

                               highway_motorway  highway_motorway_link  ...  \
u           v           key                                             ...   
29445653    4433425415  0   0                 0                      1  ...   
29445655    4433425410  0   0                 0                      1  ...   
29445656    4433425404  0   0                 0                      1  ...   
29445657    1864943659  0   0                 0                      1  ...   
29445659    4433425403  0   0                 0                      1  ...   
...                                         ...                    ...  ...   
11919304502 11919304503 0   0                 0                      0  ...   
11919304503 11919304502 0   0                 0                      0  ...   
            6585419248  0   0                 0                      0  ...   
11920190240 11920190241 0   0                 0                      0  ...   
11920190241 4273638466  0   0                 0                      0  ...   

                               junction_roundabout  junction_jughandle  \
u           v           key                                              
29445653    4433425415  0   0                    0                   0   
29445655    4433425410  0   0                    0                   0   
29445656    4433425404  0   0                    0                   0   
29445657    1864943659  0   0                    0                   0   
29445659    4433425403  0   0                    0                   0   
...                                            ...                 ...   
11919304502 11919304503 0   0                    0                   0   
11919304503 11919304502 0   0                    0                   0   
            6585419248  0   0                    0                   0   
11920190240 11920190241 0   0                    0                   0   
11920190241 4273638466  0   0                    0                   0   

                               junct

43
Index(['oneway', 'lanes', 'maxspeed', 'reversed', 'length', 'width',
       'highway_busway', 'highway_living_street', 'highway_motorway',
       'highway_motorway_link', 'highway_primary', 'highway_primary_link',
       'highway_residential', 'highway_secondary', 'highway_secondary_link',
       'highway_stop_line', 'highway_tertiary', 'highway_tertiary_link',
       'highway_trunk', 'highway_trunk_link', 'highway_unclassified',
       'access_bus', 'access_customers', 'access_destination', 'access_no',
       'access_permissive', 'access_permit', 'access_yes', 'access_residents',
       'access_designated', 'access_delivery', 'junction_circular',
       'junction_intersection', 'junction_roundabout', 'junction_jughandle',
       'junction_spui', 'bridge_movable', 'bridge_viaduct', 'bridge_yes',
       'bridge_low_water_crossing', 'tunnel_building_passage', 'tunnel_yes',
       'tunnel_no'],
      dtype='object')


#### New York

In [3]:
data = pd.read_csv('../data/downstream_tasks/airbnb/new_york/airbnb_new_york_cleaned.csv')
gdf_airbnb_new_york = gpd.GeoDataFrame(
            data.drop(["latitude", "longitude"], axis=1),
            geometry=gpd.points_from_xy(x=data["longitude"], y=data["latitude"]),
            crs="EPSG:4326",
        )

gdf_airbnb_new_york['airbnb'] = 1

In [4]:
G =ox.graph.graph_from_place("New York City, United States", network_type="drive", simplify=False)
gdf_nodes_ny, gdf_edges_ny = ox.graph_to_gdfs(G)

In [5]:
graph_embedder_ny = OSMnxGraph(gdf_airbnb_new_york, gdf_nodes_ny, gdf_edges_ny, y_column_name='airbnb')

osmnx_nodes_ny = graph_embedder_ny.get_node_attrs()
display(osmnx_nodes_ny)
print(len(osmnx_nodes_ny.columns.unique()))
print(osmnx_nodes_ny.columns.unique())
osmnx_nodes_ny.to_csv('../data/osmnx/new_york_osmnx_nodes.csv', index=False)

osmnx_edges_ny = graph_embedder_ny.get_edge_attrs(convert_to_meters=True)
display(osmnx_edges_ny)
print(len(osmnx_edges_ny.columns.unique()))
print(osmnx_edges_ny.columns.unique())
osmnx_edges_ny.to_csv('../data/osmnx/new_york_osmnx_edges.csv', index=False)

/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,bus_stop,crossing,give_way,motorway_junction,priority,speed_camera,stop,stop;crossing,street_lamp,...,traffic_signals;crossing,turning_circle,turning_loop,construction,stop_line,proposed,passing_place,mini_roundabout,milestone,red_light_camera
osmid,,,,,,,,,,,,,,,,,,,,,
33751990,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751991,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751992,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751993,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751994,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11920390759,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11920390760,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11920420149,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


22
Index(['street_count', 'bus_stop', 'crossing', 'give_way', 'motorway_junction',
       'priority', 'speed_camera', 'stop', 'stop;crossing', 'street_lamp',
       'toll_gantry', 'traffic_signals', 'traffic_signals;crossing',
       'turning_circle', 'turning_loop', 'construction', 'stop_line',
       'proposed', 'passing_place', 'mini_roundabout', 'milestone',
       'red_light_camera'],
      dtype='object')


/Users/jjaniak/Documents/studia/projekt/gradient/notebooks/../src/graph/create_osmnx_graph.py:108: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  war

oneway  lanes maxspeed  reversed  length  width  \
u           v          key                                                      
33751990    3779065560 0   0       1      4       80         0   37.56  10.54   
33751991    274185083  0   0       1      4       80         0   35.46  10.54   
33751992    274186544  0   0       1      4       80         0  105.70  10.54   
33751993    274188295  0   0       1      4       80         0  108.33  10.54   
33751994    274188831  0   0       1      4       80         0   81.85  10.54   
...                              ...    ...      ...       ...     ...    ...   
11920420149 8300030766 0   0       0      2       44         1  169.09  10.54   
11920420154 42809352   0   0       0      2       44         0    8.83  10.54   
            8300030761 0   0       0      2       44         1    3.38  10.54   
11920430504 8090944215 0   0       0      2       44         0   66.40  10.54   
            42838836   0   0       0      2       44         1    5.65  10.54   

                              highway_living_street  highway_motorway  \
u           v          key                                              
33751990    3779065560 0   0                      0                 1   
33751991    274185083  0   0                      0                 1   
33751992    274186544  0   0                      0                 1   
33751993    274188295  0   0                      0                 1   
33751994    274188831  0   0                      0                 1   
...                                             ...               ...   
11920420149 8300030766 0   0                      0                 0   
11920420154 42809352   0   0                      0                 0   
            8300030761 0   0                      0                 0   
11920430504 8090944215 0   0                      0                 0   
            42838836   0   0                      0                 0   

                              highway_motorway_link  highway_primary  ...  \
u           v          key                                            ...   
33751990    3779065560 0   0                      0                0  ...   
33751991    274185083  0   0                      0                0  ...   
33751992    274186544  0   0                      0                0  ...   
33751993    274188295  0   0                      0                0  ...   
33751994    274188831  0   0                      0                0  ...   
...                                             ...              ...  ...   
11920420149 8300030766 0   0                      0                0  ...   
11920420154 42809352   0   0                      0                0  ...   
            8300030761 0   0                      0                0  ...   
11920430504 8090944215 0   0                      0                0  ...   
            42838836   0   0                      0                0  ...   

                              junction_roundabout  junction_spui  \
u           v          key                                         
33751990    3779065560 0   0                    0              0   
33751991    274185083  0   0                    0              0   
33751992    274186544  0   0                    0              0   
33751993    274188295  0   0                    0              0   
33751994    274188831  0   0                    0              0   
...                                           ...            ...   
11920420149 8300030766 0   0                    0              0   
11920420154 42809352   0   0                    0              0   
            8300030761 0   0                    0              0   
11920430504 8090944215 0   0                    0              0   
            42838836   0   0                    0              0   

                              junction_intersection  bridge_movable  \
u           v          key                                          

43
Index(['oneway', 'lanes', 'maxspeed', 'reversed', 'length', 'width',
       'highway_living_street', 'highway_motorway', 'highway_motorway_link',
       'highway_primary', 'highway_primary_link', 'highway_residential',
       'highway_secondary', 'highway_secondary_link', 'highway_tertiary',
       'highway_tertiary_link', 'highway_trunk', 'highway_trunk_link',
       'highway_unclassified', 'highway_busway', 'highway_stop_line',
       'access_customers', 'access_designated', 'access_destination',
       'access_no', 'access_permissive', 'access_permit', 'access_yes',
       'access_residents', 'access_bus', 'access_delivery',
       'junction_circular', 'junction_jughandle', 'junction_roundabout',
       'junction_spui', 'junction_intersection', 'bridge_movable',
       'bridge_viaduct', 'bridge_yes', 'bridge_low_water_crossing',
       'tunnel_building_passage', 'tunnel_no', 'tunnel_yes'],
      dtype='object')


## PL

In [25]:
city = 'Wrocław'

df_accidents = pd.read_csv("../data/wypadki-pl/accidents.csv")
df_accidents.drop(df_accidents[(df_accidents['mie_nazwa'] != city)].index, inplace=True)
df_accidents.drop(columns='uczestnicy', inplace=True)
geometry = [Point(xy) for xy in zip(df_accidents['wsp_gps_x'], df_accidents['wsp_gps_y'])]
gdf_accidents = gpd.GeoDataFrame(df_accidents, geometry=geometry)
gdf_accidents.drop(columns=['wsp_gps_x', 'wsp_gps_y'], inplace=True)

In [26]:
cs = '[out:json][timeout:180][date:"2022-01-01T00:00:00Z"]'
ox.settings.overpass_settings = cs
G =ox.graph.graph_from_place(f"{city}, Polska", network_type="drive", simplify=False)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [27]:
graph_embedder = OSMnxGraph(gdf_airbnb_seattle, gdf_nodes_seattle, gdf_edges_seattle, y_column_name='mie_nazwa')

osmnx_nodes = graph_embedder.get_node_attrs()
display(osmnx_nodes)
print(len(osmnx_nodes.columns.unique()))
print(osmnx_nodes.columns.unique())
osmnx_nodes.to_csv(f'../data/osmnx/{city}_osmnx_nodes.csv', index=False)

osmnx_edges = graph_embedder.get_edge_attrs()
display(osmnx_edges)
print(len(osmnx_edges.columns.unique()))
print(osmnx_edges.columns.unique())
osmnx_edges.to_csv(f'../data/osmnx/{city}_osmnx_edges.csv', index=False)


/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,bus_stop,crossing,give_way,milestone,mini_roundabout,motorway_junction,red_light_camera,speed_camera,stop,...,traffic_signals,traffic_signals;crossing,turning_circle,turning_loop,street_lamp,proposed,stop;crossing,priority,construction,passing_place
osmid,,,,,,,,,,,,,,,,,,,,,
29445653,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445655,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445656,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445657,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445659,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11919304501,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11919304502,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11919304503,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


22
Index(['street_count', 'bus_stop', 'crossing', 'give_way', 'milestone',
       'mini_roundabout', 'motorway_junction', 'red_light_camera',
       'speed_camera', 'stop', 'stop_line', 'toll_gantry', 'traffic_signals',
       'traffic_signals;crossing', 'turning_circle', 'turning_loop',
       'street_lamp', 'proposed', 'stop;crossing', 'priority', 'construction',
       'passing_place'],
      dtype='object')


/Users/jjaniak/Documents/studia/projekt/gradient/notebooks/../src/graph/create_osmnx_graph.py:104: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  war

oneway  lanes maxspeed  reversed  length  \
u           v           key                                               
29445653    4433425415  0   0       1      2       97         0   20.39   
29445655    4433425410  0   0       1      2       97         0   12.41   
29445656    4433425404  0   0       1      2       97         0   14.23   
29445657    1864943659  0   0       1      2       97         0   14.65   
29445659    4433425403  0   0       1      2       97         0   12.50   
...                               ...    ...      ...       ...     ...   
11919304502 11919304503 0   0       0      2       40         1   11.38   
11919304503 11919304502 0   0       0      2       40         0   11.38   
            6585419248  0   0       0      2       40         1    4.09   
11920190240 11920190241 0   0       1      4       40         0   12.28   
11920190241 4273638466  0   0       1      4       40         0   11.09   

                               width  highway_busway  highway_living_street  \
u           v           key                                                   
29445653    4433425415  0   0  12.26               0                      0   
29445655    4433425410  0   0  12.26               0                      0   
29445656    4433425404  0   0  12.26               0                      0   
29445657    1864943659  0   0  12.26               0                      0   
29445659    4433425403  0   0  12.26               0                      0   
...                              ...             ...                    ...   
11919304502 11919304503 0   0  12.26               0                      0   
11919304503 11919304502 0   0  12.26               0                      0   
            6585419248  0   0  12.26               0                      0   
11920190240 11920190241 0   0  12.26               0                      0   
11920190241 4273638466  0   0  12.26               0                      0   

                               highway_motorway  highway_motorway_link  ...  \
u           v           key                                             ...   
29445653    4433425415  0   0                 0                      1  ...   
29445655    4433425410  0   0                 0                      1  ...   
29445656    4433425404  0   0                 0                      1  ...   
29445657    1864943659  0   0                 0                      1  ...   
29445659    4433425403  0   0                 0                      1  ...   
...                                         ...                    ...  ...   
11919304502 11919304503 0   0                 0                      0  ...   
11919304503 11919304502 0   0                 0                      0  ...   
            6585419248  0   0                 0                      0  ...   
11920190240 11920190241 0   0                 0                      0  ...   
11920190241 4273638466  0   0                 0                      0  ...   

                               junction_roundabout  junction_jughandle  \
u           v           key                                              
29445653    4433425415  0   0                    0                   0   
29445655    4433425410  0   0                    0                   0   
29445656    4433425404  0   0                    0                   0   
29445657    1864943659  0   0                    0                   0   
29445659    4433425403  0   0                    0                   0   
...                                            ...                 ...   
11919304502 11919304503 0   0                    0                   0   
11919304503 11919304502 0   0                    0                   0   
            6585419248  0   0                    0                   0   
11920190240 11920190241 0   0                    0                   0   
11920190241 4273638466  0   0                    0                   0   

                               junct

43
Index(['oneway', 'lanes', 'maxspeed', 'reversed', 'length', 'width',
       'highway_busway', 'highway_living_street', 'highway_motorway',
       'highway_motorway_link', 'highway_primary', 'highway_primary_link',
       'highway_residential', 'highway_secondary', 'highway_secondary_link',
       'highway_stop_line', 'highway_tertiary', 'highway_tertiary_link',
       'highway_trunk', 'highway_trunk_link', 'highway_unclassified',
       'access_bus', 'access_customers', 'access_destination', 'access_no',
       'access_permissive', 'access_permit', 'access_yes', 'access_residents',
       'access_designated', 'access_delivery', 'junction_circular',
       'junction_intersection', 'junction_roundabout', 'junction_jughandle',
       'junction_spui', 'bridge_movable', 'bridge_viaduct', 'bridge_yes',
       'bridge_low_water_crossing', 'tunnel_building_passage', 'tunnel_yes',
       'tunnel_no'],
      dtype='object')
